## Compare African allele frequency data on variants of interest to that of other global populations

In [95]:
# Import modules and packages

import os
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from upsetplot import plot
from requests import get, codes as http_code

In [96]:
# Set constants and functions

home_path = str(os.path.dirname(os.getcwd()))
population_clusters = ["SUPER","SUB"]

gene_location_df = pd.read_csv(os.path.join(home_path, "Data_descriptions", "locations.csv"))
genes = gene_location_df.location_name

sample_info_df = pd.read_csv(os.path.join(home_path, "Data_descriptions", "samples.csv"))
sub_populations = sample_info_df.SUB.unique()
regional_classification = {"ACB":"ACB","ASW":"ASW","GWD":"WA","ESN":"WA","MSL":"WA","MbutiPygmy":"CA","BiakaPygmy":"CA","Mandenka":"WA","Yoruba":"WA","San":"KS","BantuSouthAfrica":"SA","BantuKenya":"EA","YRI":"WA","LWK":"EA"} # SA = South Africa, CA = Central Africa, WA = West Africa, EA = East Africa, ACB = African Carribean in Barbados, ASW = African American, KS = Khoi-San

# Create a list with variant IDs of interest
NESHIE_rsIDs = ["rs2067853","rs1217401","rs2043211","rs1001179","rs1961495","rs1411040","rs34004222","rs13027659","rs190148408","rs79704487","rs1800896","rs3024490","rs1800871","rs1554286","rs1518111","rs1143623","rs16944","rs1071676","rs1800795","rs2069837","rs1800796","rs2066992","rs2069832","rs2069833","rs1554606","rs2069845","rs1801133","rs4846049","rs1476413","rs1801131","rs9651118","rs1808593","rs2070744","rs1800779","rs6517135","rs1799964","rs1799724","rs361525","rs1800629"]

# Set plot figure area and font size
sns.set(rc={"figure.figsize": (12, 10)})
SMALL_font = 8
MEDIUM_font = 10
BIGGER_font = 16

plt.rc('font', size=SMALL_font)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_font)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_font)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_font)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_font)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_font)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_font)  # fontsize of the figure title

# Functions for retrieving variant allele ALFA count information from the NCBI API

def get_ALFA_count_info(variant_id_number: str):
  """
  Retrieve ALFA allele counts from NCBI in JSON format
  """
  url = "https://api.ncbi.nlm.nih.gov/variation/v0/refsnp/{}/frequency".format(variant_id_number)
  request = get(url)
  if request.status_code != http_code.ok:
    raise Exception("Request failed: {}".format(request.status_code))
  else:
    request_json = request.json()
  return request_json

def add_all_pops(populations, project):
  """
  Add metadata to translate ALFA population code data into legible population names into a single dataframe. Code copied from https://github.com/ncbi/dbsnp/blob/master/tutorials/Variation%20Services/Jupyter_Notebook/by_rsid.ipynb.
  """  
  for p in populations:
    project['pops'][p['biosample_id']] = p
  if 'subs' in p:
    add_all_pops(p['subs'], project)

def allele_counts (study_id, study_counts):
  """
  Parse study count information and return reference and alternate allele counts. Code modified from https://github.com/ncbi/dbsnp/blob/master/tutorials/Variation%20Services/Jupyter_Notebook/by_rsid.ipynb. 
  """
  allele_counts = study_counts["allele_counts"]
  pop_allele_counts = pd.DataFrame()
  for pop_id, pop_counts in allele_counts.items():
      for allele, count in pop_counts.items():
          study_id = study_id
          pop_id = pop_id
          return ("Allele: {}, Count: {}".format(allele, count))

def get_metadata():
  """
  Retrieve ALFA allele count metadata 
  """
  url = "https://api.ncbi.nlm.nih.gov/variation/v0/metadata/frequency"
  request = get(url)
  if request.status_code != http_code.ok:
    raise Exception("Request failed: {}".format(request.status_code))
  else:
    request_json = request.json()
  return request_json

def calculate_frequencies(ref_count: int, alt_count: int):
  """
  Calculate allele frequencies from reference and alternate allele count information
  """
  total_count = ref_count + alt_count
  alt_frequency = alt_count / total_count
  ref_frequency = ref_count / total_count

  return (alt_frequency, ref_frequency)

def add_prefix_dataframe_col_names(dataframe, columns_list, prefix):
    rename_dict = dict()
    for line in columns_list:
        key = line
        if key not in rename_dict:
            rename_dict[key] = str()
        rename_dict[key] = prefix + "{}".format(line)
    dataframe = dataframe.rename(columns = rename_dict)
    return dataframe


In [97]:
# Extract global allele count data on NESHIE variant IDs from the NCBI ALFA Allele Frequency Project via API.

variant_global_allele_counts = pd.DataFrame()

for variant_id in NESHIE_rsIDs:
    variant_id_number = variant_id.replace("rs","")
    count_data = get_ALFA_count_info(variant_id_number)
    population_count_data = count_data["results"]
    for interval, data in population_count_data.items():
      variant_ref = data['ref']
      variant_study_counts = data["counts"]
      for study_code, study_allele_counts in variant_study_counts.items():
        population_counts = study_allele_counts["allele_counts"]
        variant_population_allele_count = pd.DataFrame()
        for population_code, allele_counts in population_counts.items():
          variant_population_allele_count["study_code"] = [study_code]            
          variant_population_allele_count["variant_id"] = [variant_id]
          variant_population_allele_count["population_code"] = [population_code]
          variant_population_allele_count["reference_allele"] = [variant_ref]
          variant_population_allele_count["allele_counts"] = [allele_counts]
          variant_global_allele_counts = pd.concat([variant_global_allele_counts, variant_population_allele_count])       


In [98]:
# Normalise allele count column of variant count dataframe
normalised_count_data = pd.json_normalize(variant_global_allele_counts.allele_counts)
drop_count_data = variant_global_allele_counts.drop(columns="allele_counts")
normalised_variant_count_data = pd.concat([drop_count_data.reset_index(drop=True), normalised_count_data.reset_index(drop=True)], axis = 1) 
normalised_variant_count_data  

,study_code,variant_id,population_code,reference_allele,A,G,T,C
0,PRJNA507278,rs2067853,SAMN10492705,G,42603.0,101389.0,NaN,NaN
1,PRJNA507278,rs2067853,SAMN10492695,G,39304.0,84208.0,NaN,NaN
2,PRJNA507278,rs2067853,SAMN10492703,G,434.0,6158.0,NaN,NaN
3,PRJNA507278,rs2067853,SAMN10492696,G,8.0,222.0,NaN,NaN
4,PRJNA507278,rs2067853,SAMN10492698,G,426.0,5936.0,NaN,NaN
...,...,...,...,...,...,...,...,...
463,PRJNA507278,rs1800629,SAMN10492701,G,121.0,1737.0,NaN,NaN
464,PRJNA507278,rs1800629,SAMN10492699,G,73.0,511.0,NaN,NaN
465,PRJNA507278,rs1800629,SAMN10492700,G,455.0,4911.0,NaN,NaN
466,PRJNA507278,rs1800629,SAMN10492702,G,30.0,294.0,NaN,NaN


In [99]:
# Translate population codes into legible population names.

# Fetch study population metadata 
metadata_json = get_metadata()

# Generate list of all unique study and population codes that need to be queried

study_codes = []
for study_code in normalised_variant_count_data.study_code.unique(): 
    study_codes.append(study_code)

population_codes = []
for population_code in normalised_variant_count_data.population_code.unique():
    population_codes.append(population_code)

# Fetch study and population name metadata. Code modified from https://github.com/ncbi/dbsnp/blob/master/tutorials/Variation%20Services/Jupyter_Notebook/by_rsid.ipynb.
metadata = {}
for project_json in metadata_json:
  p = {}
  p['json']=project_json
  p['pops']={}
  metadata[project_json['bioproject_id']] = p
for prj_id, prj in metadata.items():
  add_all_pops(prj['json']['populations'], prj)

for study_id in study_codes:
    study_name = metadata[study_id]['json']['short_name']

    pop_dict = {"SAMN10492696" : "African Others", "SAMN10492698" : "African American", "SAMN10492697" : "East Asian", "SAMN10492701" : "Other Asian"}
    for pop_id in population_codes: 
        if pop_id not in ["SAMN10492696", "SAMN10492698", "SAMN10492697", "SAMN10492701"]:
            population_name = metadata[study_id]['pops'][pop_id]['name']
            temp_pop_dict = {pop_id : population_name}
        pop_dict.update(temp_pop_dict)

# Add column in population allele count dataframe for population names

normalised_variant_count_data["Population"] = normalised_variant_count_data["population_code"].map(pop_dict)
normalised_variant_count_data

,study_code,variant_id,population_code,reference_allele,A,G,T,C,Population
0,PRJNA507278,rs2067853,SAMN10492705,G,42603.0,101389.0,NaN,NaN,Total
1,PRJNA507278,rs2067853,SAMN10492695,G,39304.0,84208.0,NaN,NaN,European
2,PRJNA507278,rs2067853,SAMN10492703,G,434.0,6158.0,NaN,NaN,African
3,PRJNA507278,rs2067853,SAMN10492696,G,8.0,222.0,NaN,NaN,African Others
4,PRJNA507278,rs2067853,SAMN10492698,G,426.0,5936.0,NaN,NaN,African American
...,...,...,...,...,...,...,...,...,...
463,PRJNA507278,rs1800629,SAMN10492701,G,121.0,1737.0,NaN,NaN,Other Asian
464,PRJNA507278,rs1800629,SAMN10492699,G,73.0,511.0,NaN,NaN,Latin American 1
465,PRJNA507278,rs1800629,SAMN10492700,G,455.0,4911.0,NaN,NaN,Latin American 2
466,PRJNA507278,rs1800629,SAMN10492702,G,30.0,294.0,NaN,NaN,South Asian


In [100]:
# Calculate allele frequencies from allele count data and append to allele count dataframe
normalised_variant_count_data["total_count"] = normalised_variant_count_data[["G", "C", "A", "T"]].sum(axis=1)

normalised_variant_count_data["G_freq"] = normalised_variant_count_data["G"] / normalised_variant_count_data["total_count"]
normalised_variant_count_data["C_freq"] = normalised_variant_count_data["C"] / normalised_variant_count_data["total_count"]
normalised_variant_count_data["A_freq"] = normalised_variant_count_data["A"] / normalised_variant_count_data["total_count"]
normalised_variant_count_data["T_freq"] = normalised_variant_count_data["T"] / normalised_variant_count_data["total_count"]

ALFA_freq_data = normalised_variant_count_data.drop(columns=["G", "C", "A", "T", "total_count", "study_code", "population_code"])
ALFA_freq_data

,variant_id,reference_allele,Population,G_freq,C_freq,A_freq,T_freq
0,rs2067853,G,Total,0.704129,NaN,0.295871,NaN
1,rs2067853,G,European,0.681780,NaN,0.318220,NaN
2,rs2067853,G,African,0.934163,NaN,0.065837,NaN
3,rs2067853,G,African Others,0.965217,NaN,0.034783,NaN
4,rs2067853,G,African American,0.933040,NaN,0.066960,NaN
...,...,...,...,...,...,...,...
463,rs1800629,G,Other Asian,0.934876,NaN,0.065124,NaN
464,rs1800629,G,Latin American 1,0.875000,NaN,0.125000,NaN
465,rs1800629,G,Latin American 2,0.915207,NaN,0.084793,NaN
466,rs1800629,G,South Asian,0.907407,NaN,0.092593,NaN


In [101]:
# Import in-house allele count data

collated_count_data = pd.DataFrame()
for gene in genes:
    for sub_population in sub_populations:
        allele_count_path = os.path.join(home_path,"Data", "SUB", "ALL_{}.{}.acount".format(gene, sub_population))
        if os.path.exists(allele_count_path):
            allele_count_df = pd.read_csv(allele_count_path, sep="\t")
            allele_count_df["REF_CTS"] = allele_count_df["OBS_CT"] - allele_count_df["ALT_CTS"]
            allele_count_df["GENE"] = gene
            allele_count_df["SUB_POP"] = sub_population
        collated_count_data = pd.concat([collated_count_data, allele_count_df]).drop(columns="OBS_CT")

# Filter in-house count data for variants of interest
collated_count_subset = collated_count_data[collated_count_data.ID.isin(NESHIE_rsIDs)]
collated_count_subset["REG"] = collated_count_subset["SUB_POP"].map(regional_classification)
collated_count_subset


C:\Users\User\AppData\Local\Temp\ipykernel_21596\2094901536.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  collated_count_subset["REG"] = collated_count_subset["SUB_POP"].map(regional_classification)


,#CHROM,ID,REF,ALT,ALT_CTS,REF_CTS,GENE,SUB_POP,REG
3716,13,rs34004222,G,A,0,188,COL4A1,ACB,ACB
4941,13,rs1961495,C,T,34,154,COL4A1,ACB,ACB
8871,13,rs1411040,C,T,161,27,COL4A1,ACB,ACB
3716,13,rs34004222,G,A,0,232,COL4A1,GWD,WA
4941,13,rs1961495,C,T,15,217,COL4A1,GWD,WA
...,...,...,...,...,...,...,...,...,...
259,1,rs4846049,T,G,51,53,MTHFR,ASW,ASW
382,1,rs1476413,C,T,16,88,MTHFR,ASW,ASW
542,1,rs1801131,T,G,20,84,MTHFR,ASW,ASW
623,1,rs1801133,G,A,16,88,MTHFR,ASW,ASW


In [111]:
# Calculate in-house data allele counts for the total African population, the total African population excluding non-native African American and Carribean Africans, the total African population excluding Khoi-San people, and Southern/South Africa with the addition of Khoi-San populations. Append the additional allele frequency data to the existing allele frequency dataframe. 
collated_count_subset_b = collated_count_subset[["ID","REF","ALT","ALT_CTS","REF_CTS","REG"]].copy()
total_africa_ct = collated_count_subset_b.groupby(["ID", "REF", "ALT"]).sum(numeric_only=True).reset_index().assign(REG = 'African')
native_africa_ct = collated_count_subset_b[(collated_count_subset.REG != "ACB") & (collated_count_subset.REG != "ASW")].groupby(["ID", "REF", "ALT"]).sum(numeric_only=True).reset_index().assign(REG = 'Native African')
native_africa_no_ks_ct = collated_count_subset_b[(collated_count_subset.REG != "KS") & (collated_count_subset.REG != "ACB") & (collated_count_subset.REG != "ASW")].groupby(["ID", "REF", "ALT"]).sum(numeric_only=True).reset_index().assign(REG = 'Native African without KS')
SA_KS_ct = collated_count_subset_b[(collated_count_subset.REG == "SA") | (collated_count_subset.REG == "KS")].groupby(["ID", "REF", "ALT"]).sum(numeric_only=True).reset_index().assign(REG = 'SA and KS')

collated_count_data = pd.concat([collated_count_subset_b, total_africa_ct, native_africa_ct, SA_KS_ct, native_africa_no_ks_ct]).sort_values("ID").reset_index(drop=True)
collated_count_data

,ID,REF,ALT,ALT_CTS,REF_CTS,REG
0,rs1071676,C,G,4,8,KS
1,rs1071676,C,G,2,14,SA
2,rs1071676,C,G,212,1298,African
3,rs1071676,C,G,26,162,ACB
4,rs1071676,C,G,173,1045,Native African
...,...,...,...,...,...,...
517,rs9651118,T,C,0,1218,Native African
518,rs9651118,T,C,0,166,WA
519,rs9651118,T,C,0,206,WA
520,rs9651118,T,C,0,42,WA


In [112]:
# Generate list of all unique regions in collated_count_data dataframe

inhouse_populations = []
for region in set(collated_count_data.REG.values):
    inhouse_populations.append(region)

inhouse_populations

['SA',
 'Native African',
 'ASW',
 'KS',
 'ACB',
 'African',
 'SA and KS',
 'WA',
 'Native African without KS',
 'EA',
 'CA']

In [104]:
# Format in-house allele count data in format suitable for Fisher's test

ih_fishers_data = collated_count_data[collated_count_data.ID != "."] # Drop ambiguous variant IDs
ih_fishers_data = ih_fishers_data.drop_duplicates(["ID","REF","ALT","REG"]) # Drop duplicate entries
ih_fishers_data = ih_fishers_data.pivot(index=["ID","REF","ALT"], columns="REG", values=["ALT_CTS","REF_CTS"]) # Pivot data

# Separate alternate and reference count data into different dataframes to facilate renaming of count columns appropriately
ih_fishers_data_alt = ih_fishers_data[['ALT_CTS']].droplevel(level=0, axis=1).reset_index() 
fishers_data_ref = ih_fishers_data[['REF_CTS']].droplevel(level=0, axis=1).reset_index()

ih_fishers_data_alt = add_prefix_dataframe_col_names(fishers_data_ref, inhouse_populations,"IH_ALT_CT_")
ih_fishers_data_ref = add_prefix_dataframe_col_names(fishers_data_ref, inhouse_populations,"IH_REF_CT_")

# Merge renamed alternate and reference count data
ih_fishers_data_renamed = ih_fishers_data_alt.merge(ih_fishers_data_ref, on=["ID","REF","ALT"])
ih_fishers_data_renamed

REG,ID,REF,ALT,IH_ALT_CT_ACB,IH_ALT_CT_ASW,IH_ALT_CT_African,IH_ALT_CT_CA,IH_ALT_CT_EA,IH_ALT_CT_KS,IH_ALT_CT_Native African,...,IH_REF_CT_ASW,IH_REF_CT_African,IH_REF_CT_CA,IH_REF_CT_EA,IH_REF_CT_KS,IH_REF_CT_Native African,IH_REF_CT_Native African without KS,IH_REF_CT_SA,IH_REF_CT_SA and KS,IH_REF_CT_WA
0,rs1071676,C,G,162,91,1298,22,163,8,1045,...,91,1298,22,163,8,1045,1037,14,22,184
1,rs1217401,A,G,45,25,317,17,5,4,247,...,25,317,17,5,4,247,243,3,7,39
2,rs1411040,C,T,27,13,264,11,7,1,224,...,13,264,11,7,1,224,223,7,8,21
3,rs1476413,C,T,154,88,1287,27,18,8,1045,...,88,1287,27,18,8,1045,1037,12,20,176
4,rs1518111,T,C,71,38,650,6,6,1,541,...,38,650,6,6,1,541,540,5,6,108
5,rs1554286,A,G,69,38,646,13,72,1,539,...,38,646,13,72,1,539,538,5,6,22
6,rs1554606,T,G,67,34,473,13,5,6,372,...,34,473,13,5,6,372,366,3,9,12
7,rs1800779,G,A,24,17,217,5,2,0,176,...,17,217,5,2,0,176,176,3,3,28
8,rs1800795,C,G,12,9,22,0,0,0,1,...,9,22,0,0,0,1,1,0,0,1
9,rs1800796,G,C,171,97,1354,42,16,11,1086,...,97,1354,42,16,11,1086,1075,14,25,153


In [105]:
ih_fishers_data_renamed['REF']

0     C
1     A
2     C
3     C
4     T
5     A
6     T
7     G
8     C
9     G
10    A
11    T
12    T
13    G
14    G
15    C
16    A
17    G
18    G
19    A
20    C
21    C
22    A
23    G
24    C
25    A
26    G
27    T
28    T
Name: REF, dtype: object

In [109]:
# Extract ALFA data alternate allele information that corresponds with that of the in-house alternate allele count data

# Iterate through in-house variant data and extract variant ID, alternate allele and regional information:
corresponding_ALFA_count_df = pd.DataFrame()
for index, row in ih_fishers_data_renamed.drop_duplicates(subset=["ID", "REF", "ALT"]).iterrows():
    variant_ID = row["ID"]
    variant_ref = row["REF"]
    variant_alt = row["ALT"]
    
    # Extract corresponding alternate allele information from ALFA count dataframe:
    corresponding_ALFA_pop_count_df = pd.DataFrame()
    ALFA_populations = []
    for population in set(normalised_variant_count_data.Population.values): # Loop through every unique population in the ALFA count dataframe
        ALFA_populations.append(population)
        corresponding_ALFA_pop_count = normalised_variant_count_data[(normalised_variant_count_data.variant_id == variant_ID) & (normalised_variant_count_data.reference_allele == variant_ref) & (normalised_variant_count_data.Population == population)] # Extract count information for a particular variant in a particular population
        corresponding_ALFA_pop_count_df["ID"] = corresponding_ALFA_pop_count.variant_id.values
        corresponding_ALFA_pop_count_df["REG"] = corresponding_ALFA_pop_count.Population.values
        corresponding_ALFA_pop_count_df["REF"] = corresponding_ALFA_pop_count.reference_allele.values
        corresponding_ALFA_pop_count_df["ALT"] = variant_alt
        corresponding_ALFA_pop_count_df["ALFA_REF_CTS"] = corresponding_ALFA_pop_count[variant_ref].values
        corresponding_ALFA_pop_count_df["ALFA_ALT_CTS"] = corresponding_ALFA_pop_count[variant_alt].values
        
        corresponding_ALFA_count_df = pd.concat([corresponding_ALFA_count_df, corresponding_ALFA_pop_count_df]).reset_index(drop=True)

corresponding_ALFA_count_pivot = corresponding_ALFA_count_df.pivot(index=["ID","REF","ALT"], columns="REG", values=["ALFA_REF_CTS","ALFA_ALT_CTS"]) # Pivot data

corresponding_ALFA_count_alt = corresponding_ALFA_count_pivot[['ALFA_ALT_CTS']].droplevel(level=0, axis=1).reset_index() 
corresponding_ALFA_count_ref = corresponding_ALFA_count_pivot[['ALFA_REF_CTS']].droplevel(level=0, axis=1).reset_index()
        
ALFA_data_alt = add_prefix_dataframe_col_names(corresponding_ALFA_count_alt, ALFA_populations,"ALFA_ALT_CT_")
ALFA_data_ref = add_prefix_dataframe_col_names(corresponding_ALFA_count_ref, ALFA_populations,"ALFA_REF_CT_")

ALFA_data = pd.concat([ALFA_data_ref, ALFA_data_alt], axis=1)
print (ALFA_data)

REG          ID REF ALT  ALFA_REF_CT_African  ALFA_REF_CT_African American  \
0     rs1071676   C   G               2489.0                        2392.0   
1     rs1217401   A   G               3779.0                        3697.0   
2     rs1411040   C   T               1218.0                        1169.0   
3     rs1476413   C   T               7510.0                        7223.0   
4     rs1518111   T   C               3285.0                        3154.0   
5     rs1554286   A   G               3188.0                        3069.0   
6     rs1554606   T   G               1716.0                        1651.0   
7     rs1800779   G   A                581.0                         568.0   
8     rs1800795   C   G                195.0                         194.0   
9     rs1800796   G   C               2657.0                        2552.0   
10    rs1800871   A   G               3248.0                        3117.0   
11    rs1800896   T   C               5586.0                    